In [12]:
%%capture
# comment the previous line if experiencing package issues and want to debug
%pip install -r requirements.txt

# Generative AI for Biomedical Decisions

In [13]:
import os
import json
import requests

from openai import OpenAI

%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [14]:
# Just makes outputs formatted a little better
from IPython.display import Markdown, display

def printmd(string: str):
    display(Markdown(string))

## Getting Started: Generate Responses with Open AI

The `OpenAI` class creates a Python object that you can then use to generate responses from any of their models. For the most part, other models use a similar API where you call class methods when generating outputs. Prior to running this cell, make sure that you have created a `.env` file containing your `OPENAI_API_KEY`.

In [15]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

Responses are generated using the `create()` method. A [full list of parameters can be found here](https://github.com/openai/openai-python/blob/main/api.md#responses). If you are creating an application, look [using streaming respones](https://platform.openai.com/docs/guides/streaming-responses?api-mode=chat).

In [16]:
user_prompt = "How do I administer CPR?"

response = client.responses.create(
    model="gpt-4.1-nano",
    input=user_prompt,
)

printmd(response.output_text)

Administering CPR (Cardiopulmonary Resuscitation) can save a person's life if they've stopped breathing or their heart has stopped. Here's a step-by-step guide on how to perform adult CPR:

**Before you start:**
- Ensure the scene is safe.
- Check if the person is responsive by gently shaking their shoulder and shouting, "Are you okay?"
- Call emergency services immediately or ask someone else to do so.
- If you're alone and the person is unresponsive, perform CPR for about 2 minutes before calling emergency services.

**Steps for adult CPR:**

1. **Check for responsiveness and breathing:**
   - Gently tap the person and shout, "Are you okay?"
   - Look for normal breathing (not gasping) for no more than 10 seconds.

2. **Call for help:**
   - If unresponsive and not breathing normally, call emergency services or have someone do it.
   - If possible, get an Automated External Defibrillator (AED).

3. **Start chest compressions:**
   - Kneel beside the person's chest.
   - Place the heel of one hand on the center of the chest (lower half of the breastbone).
   - Place your other hand on top, interlocking your fingers.
   - Keep your arms straight and shoulders directly over your hands.
   - Push hard and fast:
     - Compress the chest at least 2 inches (5 cm) deep.
     - Use a rate of at least 100 to 120 compressions per minute (like the beat of the song "Staying Alive").

4. **Deliver rescue breaths (if trained and comfortable):**
   - After 30 compressions, give 2 rescue breaths.
   - Tilt the person's head back slightly to open the airway.
   - Pinch the nose shut, cover their mouth with yours, and blow to make the chest rise.
   - Each breath should last about 1 second, and the chest should rise visibly.

5. **Continue cycles:**
   - Keep performing cycles of 30 compressions and 2 breaths.
   - Do not stop unless:
     - Someone with more advanced training takes over.
     - The person shows signs of life.
     - You are too exhausted to continue.
     - The scene becomes unsafe.

**Using an AED:**
- Turn on the AED and follow its voice prompts.
- Attach pads to the person's bare chest as indicated.
- Ensure no one is touching the person during rhythm analysis and shock delivery.
- Follow AED instructions.

**Important notes:**
- Hands-only CPR (just chest compressions) is recommended if you're not trained or uncomfortable giving rescue breaths.
- Early defibrillation greatly improves survival chances.

If you're interested in detailed training, consider taking a certified CPR course from organizations like the American Heart Association or Red Cross.

**Remember:** Acting quickly and confidently can make a critical difference in saving a life.

You can also get responeses if wish to input an image instead. Take a look at [the OpenAI documentation](https://platform.openai.com/docs/overview) for a full list of all possible use-cases.

In [17]:
user_prompt = "What is this image"
image_url = "https://upload.wikimedia.org/wikipedia/commons/9/92/EB1911_Heart_-_Thoracic_Viscera.png"

response = client.responses.create(
    model="gpt-4.1-nano",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": user_prompt},
                {"type": "input_image", "image_url": f"{image_url}"},
            ],
        }
    ],
)

printmd(response.output_text)

This image appears to be an anatomical illustration of the human heart and lungs, with detailed labeling and structure. It is likely a historical or scientific diagram used to study or teach about the anatomy of the chest, lungs, and heart.

## Customizing Model Responses

### System Instructions

If you take a look at the response from our prompt `How do I administer CPR?`, it seems quite wordy. Also, if I were a professional in the field, I already know a number of these things.  We can simply create a set of system instructions to make the model respond more like we want.

In [7]:
system_instructions = """
You are an expert emergency physician with decades of experience in CPR. The user is also a clinician.
Keep responses under 100 words.
"""
user_prompt = "How do I administer CPR?"

response = client.responses.create(
    model="gpt-4.1-nano",
    instructions=system_instructions,
    input=user_prompt,
)

printmd(response.output_text)

Start by confirming unresponsiveness and calling 911. Place the patient on a firm surface, open the airway, and check for breathing and pulse (wart or carotid) for no more than 10 seconds. If absent or abnormal, begin CPR: **30 compressions** at a depth of about 2 inches for adults, at a rate of 100-120/min, allowing recoil. Follow with **2 rescue breaths**—pinch the nose, seal the mouth, and deliver a breath lasting 1 second, watching for chest rise. Continue until help arrives or the patient recovers.

With just a few system instructions, I was able to completely change the model to perform in a much more precise, direct, and concise manner. This is known as **prompt engineering** and can be a very powerful tool for changing model performance. [This guide](https://www.promptingguide.ai/) is a fantastic resource to get started.

### Temperature, Top P

Let's take a look at repeated responses from our model.

In [8]:
system_instructions = """
You are an expert emergency physician with decades of experience in CPR. The user is also a clinician.
Keep responses under 100 words.
"""
user_prompt = "How do I administer CPR?"

for i in range(3):
    response = client.responses.create(
        model="gpt-4.1-nano",
        instructions=system_instructions,
        input=user_prompt,
    )

    printmd(f"#### Response {i+1} Output:")
    printmd(response.output_text)
    print()

#### Response 1 Output:

Ensure safety, then check responsiveness and breathing. Call emergency services. Initiate high-quality chest compressions at 100-120 per minute, pressing hard and fast (about 2 inches deep). Allow full chest recoil between compressions. Provide rescue breaths if trained: 30 compressions to 2 breaths. Continue until help arrives or the patient recovers. Use an AED as soon as available, following prompts.

#### Response 2 Output:

Check for responsiveness and breathing; if unresponsive and not breathing normally, call 911. Begin chest compressions at 2 inches depth at a rate of 100-120 per minute. Compress the center of the chest with straight arms, allowing full recoil. If trained and confident, give rescue breaths at 30:2 ratio (30 compressions, 2 breaths). Continue until advanced help arrives, the person shows signs of life, or you are exhausted. Use an AED as soon as available.

#### Response 3 Output:

Check responsiveness and breathing. Call for emergency help. Start high-quality chest compressions: place hands on the center of the chest, compress at 2 inches depth and 100-120/min rate. Allow full recoil. For adults, deliver 30 compressions then 2 rescue breaths (if trained in CPR). For rescue breaths, tilt the head, lift the chin, pinch the nose, and give two 1-second breaths. Continue cycles until help arrives or the patient recovers. Use an AED as soon as available.

Notice that responses are a little bit different each time. That is because when the model chooses which token to use next, there is a degree of randomness involved. This can be modified by two parameters:

* **Temperature**: This skews the next token distribution towards the most likely token (low) or flattens the distribution (high). The values range from 0 to 2. Low temperatures makes responses more deterministic and high makes responses more creative.
* **Top P**: Only considers top tokens summing to cumulative probability $p$. Values range from 0 to 1. Low top P values focus only on most probable tokens while high focusses on all possible tokens.

It is generally recommended to choose either temperature or top P when controlling randomness. For medical purposes, it's recommended to keep these values low. Let's see what happens when we do that.

In [9]:
system_instructions = """
You are an expert emergency physician with decades of experience in CPR. The user is also a clinician.
Keep responses under 100 words.
"""
user_prompt = "How do I administer CPR?"

for i in range(3):
    response = client.responses.create(
        model="gpt-4.1-nano",
        instructions=system_instructions,
        input=user_prompt,
        temperature=0,
    )

    printmd(f"#### Response {i+1} Output:")
    printmd(response.output_text)
    print()

#### Response 1 Output:

Ensure scene safety, check responsiveness, and call for help. If unresponsive and no breathing or abnormal breathing, start CPR: 

1. **Chest Compressions:** Place hands on the center of the chest, compress at least 2 inches deep at 100-120 per minute.
2. **Rescue Breaths:** After 30 compressions, give 2 breaths if trained and comfortable, each lasting 1 second, watching for chest rise.
3. **Continue:** Repeat cycles of 30 compressions and 2 breaths until help arrives or the patient recovers. Use an AED as soon as available.

#### Response 2 Output:

Ensure scene safety, check responsiveness, and call for help. If unresponsive and no breathing or abnormal breathing, start CPR: 

1. **Chest Compressions:** Place hands on the center of the chest, compress at least 2 inches deep at 100-120 per minute.
2. **Rescue Breaths:** After 30 compressions, give 2 breaths if trained and comfortable, each lasting 1 second, watching for chest rise.
3. **Continue:** Repeat cycles of 30 compressions and 2 breaths until help arrives or the patient recovers. Use an AED as soon as available.

#### Response 3 Output:

Ensure scene safety, check responsiveness, and call for help. If unresponsive and no breathing or abnormal breathing, start CPR: 

1. **Chest Compressions:** Place hands on the center of the chest, compress at least 2 inches deep at 100-120 per minute.
2. **Rescue Breaths:** After 30 compressions, give 2 breaths if trained and comfortable, each lasting 1 second, watching for chest rise.
3. **Continue:** Repeat cycles of 30 compressions and 2 breaths until help arrives or the patient recovers. Use an AED as soon as available.

While not exactly the same (there is still some degree of randomness inherently within the model), for the most part responses are much more similar than they were before.

### Tool Calling

Sometimes, you may have a specific function that you would like to call that performs a specific task that supplments the LLM's performance. This may involve querying an API and getting a reponse or running a deterministic model on data you recieve from the LLM. In these cases, you will want to use a tool call. Consider an example where you want to know the weather in San Antonio, TX. You would do the following.

In [18]:
# Step 0: Create tool call function
def get_weather(latitude, longitude):
    """This function get's the weather from the lattitude and longititude of the provided location"""
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m&temperature_unit=fahrenheit")
    data = response.json()
    return data['current']['temperature_2m']

# Step 1: Call model with get_weather tool defined
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in Fahrenheit.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]
input_messages = [{"role": "user", "content": "What is the weather like in San Antonio, TX today?"}]

response = client.responses.create(
    model="gpt-4.1-nano",
    input=input_messages,
    tools=tools
)

# Step 2: Execute get_weather function
tool_call = response.output[0]
args = json.loads(tool_call.arguments)

result = get_weather(args["latitude"], args["longitude"])

# Step 3: Supply result and call model again
input_messages.append(tool_call)                 # append model's function call message
input_messages.append({                          # append result message
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

response_2 = client.responses.create(
    model="gpt-4.1-nano",
    input=input_messages,
    tools=tools,
)
print(response_2.output_text)

The current temperature in San Antonio, TX is approximately 89.3°F.


## Reasoning

Reasoning models have been built that are trained with reinforcment learning. These models have been shown to excel in harder problems such as STEM related fields. OpenAI has built the o-series of models for this task. Consider this example.

In [19]:
prompt = """
What are three compounds we should consider investigating to 
advance research into new antibiotics? Why should we consider 
them?
"""

response = client.responses.create(
    model="o4-mini",
    reasoning={"effort": "high"},
    input=[
        {
            "role": "user", 
            "content": prompt
        }
    ]
)

printmd(response.output_text)

Here are three recent lead‐compounds that look especially promising—and why they deserve serious follow-up:

1. Teixobactin  
  • Source & scaffold: an unusual non-ribosomal depsipeptide discovered from an “uncultured” soil bacterium (Eleftheria terrae) using the iChip technology.  
  • Mechanism: binds lipid II and lipid III precursors in the Gram-positive cell wall pathway—so far no resistant mutants have been raised in the lab.  
  • Why investigate?  
    – Potent in vivo efficacy against MRSA, VRE and drug-resistant Mycobacteria  
    – Novel scaffold for synthetic/semisynthetic optimization  
    – Demonstrates that the uncultured majority of soil bacteria remain a rich reservoir of new chemistry  

2. Darobactin  
  • Source & scaffold: a ribosomally-synthesized and post-translationally modified heptapeptide (RiPP) from the nematode-symbiont Photorhabdus.  
  • Mechanism: binds BamA, the essential outer-membrane protein-assembly factor in Gram-negatives, blocking membrane biogenesis.  
  • Why investigate?  
    – Potent, narrow-nanomolar activity vs. E. coli, Klebsiella, Pseudomonas, Acinetobacter  
    – Low cytotoxicity, good pharmacokinetics and in vivo protection in mouse septicemia models  
    – Novel outer-membrane target with no cross-resistance to existing Gram-negative drugs  

3. Pseudouridimycin (PUM)  
  • Source & scaffold: a nucleoside-analog natural product from Streptomyces spp.  
  • Mechanism: mimics UTP and binds the active site of bacterial RNA polymerase—completely distinct from rifamycins or lipiarmycins.  
  • Why investigate?  
    – Broad-spectrum Gram-positive activity (and potential for Gram-negative analogs with improved uptake)  
    – No cross-resistance with existing RNAP inhibitors  
    – Chemotype readily amenable to medicinal-chemistry optimization of potency, spectrum and PK  

Taken together, these three—each from a very different source family and each hitting a new essential target—represent high-value starting points for:  
  • Exploring new microbial resources (iChip, symbionts, under-studied Streptomyces)  
  • Uncovering novel mechanisms of action that sidestep existing resistance pathways  
  • Developing synthetic/semisynthetic analogs with optimized drug-like properties

## Conclusion

I hope you enjoyed this brief introduction to some of the capabilities of these AI Models and how they can be used for biomedical decision making. More information about advanced use case can be found in [OpenAI's documentation](https://platform.openai.com/docs/overview) or in the documentation website of your desired model. If you want to learn how these models work from an architecture perspective, [take a look at this tutorial](https://jalammar.github.io/illustrated-transformer/).